#EffiV1

In [1]:
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
notebook_path = '/content/drive/Shared drives/Desarrollo/'

In [ ]:
!ls "{notebook_path}"

 _annotations.csv
'Baumann, Leslie - The Skin Type Solution_ a Revolutionary Guide to Your Best Skin Ever (2010, Random House Publishing Group) - libgen.li.pdf'
 book-products
'Facial Skin.v1i.voc'
 facial_skin.zip
'Fichas tecnicas'
 files
'fotos de prueba'
 Modelo_efficientnet.ipynb
 Modelo_Unet.ipynb
 models
 README.dataset.txt
 README.roboflow.txt
 routines
 scrape
 scraped-products
 skin_defects.csv
 skin_deffects_angles.zip
 Skin_deffects.ipynb
 train
 transformer
 valid


In [ ]:
#!pip install timm
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
class SkinDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = dataframe['filename'].values
        self.labels = dataframe.drop(columns=['filename', 'source']).values.astype('float32')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx])
        return image, label

In [ ]:
notebook_path = '/content/drive/Shared drives/Desarrollo'
df = pd.read_csv(f"{notebook_path}/_annotations.csv")

df['filename'] = df['filename'].str.strip()  # Asegura que no hay espacios raros

df_train = df[df['source'] == 'train'].reset_index(drop=True)
df_test = df[df['source'] == 'test'].reset_index(drop=True)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = SkinDataset(df_train, f"{notebook_path}/train", transform=transform)
test_dataset = SkinDataset(df_test, f"{notebook_path}/valid", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = df_train.drop(columns=['filename', 'source']).shape[1]

model = timm.create_model('tf_efficientnet_b0', pretrained=True, num_classes=num_classes)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [ ]:
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    running_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)


def evaluate(model, dataloader, device):
    model.eval()
    all_labels = []
    all_preds = []
    running_loss = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            preds = torch.sigmoid(outputs).cpu().numpy()
            labels = labels.cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds)
            running_loss += loss.item()

    all_labels = np.array(all_labels)
    all_preds_bin = (np.array(all_preds) > 0.5).astype(int)

    f1 = f1_score(all_labels, all_preds_bin, average='macro')
    acc = accuracy_score(all_labels, all_preds_bin)
    return running_loss / len(dataloader), f1, acc


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nÉpoca {epoch+1}/{num_epochs}")
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    print(f"🔧 Train Loss: {train_loss:.4f}")


Época 1/10
🔧 Train Loss: 0.5323

Época 2/10
🔧 Train Loss: 0.2416

Época 3/10
🔧 Train Loss: 0.1677

Época 4/10
🔧 Train Loss: 0.1372

Época 5/10
🔧 Train Loss: 0.1126

Época 6/10
🔧 Train Loss: 0.0871

Época 7/10
🔧 Train Loss: 0.0735

Época 8/10
🔧 Train Loss: 0.0602

Época 9/10
🔧 Train Loss: 0.0504

Época 10/10
🔧 Train Loss: 0.0387


In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, hamming_loss

def evaluate_with_metrics(model, dataloader, device, class_names=None):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.sigmoid(outputs).cpu().numpy()
            labels = labels.cpu().numpy()
            all_preds.extend(probs)
            all_labels.extend(labels)

    all_preds_bin = (np.array(all_preds) > 0.5).astype(int)
    all_labels = np.array(all_labels)

    print("Classification Report (per class):")
    print(classification_report(all_labels, all_preds_bin, target_names=class_names, zero_division=0))

    print("\nOther multilabel metrics:")
    print(f"Subset accuracy: {accuracy_score(all_labels, all_preds_bin):.4f}")
    print(f"F1 macro: {f1_score(all_labels, all_preds_bin, average='macro'):.4f}")
    print(f"F1 micro: {f1_score(all_labels, all_preds_bin, average='micro'):.4f}")
    print(f"Precision macro: {precision_score(all_labels, all_preds_bin, average='macro'):.4f}")
    print(f"Recall macro: {recall_score(all_labels, all_preds_bin, average='macro'):.4f}")
    print(f"Hamming loss: {hamming_loss(all_labels, all_preds_bin):.4f}")


In [ ]:
class_names = df_train.drop(columns=['filename', 'source']).columns.tolist()
evaluate_with_metrics(model, test_loader, device, class_names=class_names)

Classification Report (per class):
              precision    recall  f1-score   support

     eye_bag       0.00      0.00      0.00         2
   acne_scar       0.07      1.00      0.13         1
     freckle       0.93      0.93      0.93        15
     nodules       0.40      0.33      0.36         6
     papules       0.52      0.52      0.52        21
    pustules       0.33      0.14      0.20         7
 skinredness       0.00      0.00      0.00         7
    vascular       0.00      0.00      0.00         3
     wrinkle       0.90      1.00      0.95        18
 dark_circle       1.00      0.56      0.72        16
  black_head       0.82      0.74      0.78        19
  white_head       0.79      0.62      0.70        24
   dark_spot       0.00      0.00      0.00         0
      stains       0.75      0.82      0.78        11

   micro avg       0.69      0.63      0.66       150
   macro avg       0.47      0.48      0.43       150
weighted avg       0.70      0.63      0.65  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#print(model.state_dict())

In [ ]:
import os
import torch
model_dir = '/content/drive/Shared drives/Desarrollo/models'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Created directory: {model_dir}")

torch.save(model.state_dict(), os.path.join(model_dir, 'efficientnet_v1.pth'))

Created directory: /content/drive/Shared drives/Desarrollo/models


# Prueba EffiV1

In [ ]:
from torchvision import transforms
from PIL import Image
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
image_path = '/content/drive/Shared drives/Desarrollo/fotos de prueba/papita.jpg'
image = Image.open(image_path).convert('RGB')


input_tensor = transform(image).unsqueeze(0).to(device)


In [ ]:
model.eval()
with torch.no_grad():
    output = model(input_tensor)
    probs = torch.sigmoid(output).cpu().numpy()[0]

In [ ]:
classes = df.drop(columns=['filename', 'source']).columns.tolist()

In [ ]:
threshold = 0.5
predicted_labels = [cls for cls, prob in zip(classes, probs) if prob >= threshold]

print("Predicciones:", predicted_labels)

Predicciones: ['acne_scar', 'freckle', 'vascular', 'stains']


#Effi V2

In [ ]:
from torchvision import transforms
import random

from torchvision import transforms

# Transformación normal (sin augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Transformación con augmentation
aug_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.02),
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
class CustomMultiLabelDataset(Dataset):
    def __init__(self, df, image_dir, classes, transform=None, aug_transform=None, minority_classes=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.aug_transform = aug_transform
        self.classes = classes
        self.minority_classes = minority_classes or []

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')

        labels = torch.tensor(row[self.classes].values.astype(np.float32))

        # Si tiene alguna clase minoritaria, aplicar augmentation
        if any(row[cls] == 1 for cls in self.minority_classes):
            if self.aug_transform:
                image = self.aug_transform(image)
        else:
            if self.transform:
                image = self.transform(image)

        return image, labels

    def __len__(self):
        return len(self.df)

In [ ]:
minority_classes = [
    'eye_bag', 'dark_spot', 'vascular', 'skinredness',
    'pustules', 'dark_circle', 'nodules', 'freckle', 'stains'
]


In [ ]:
from torch.utils.data import DataLoader

# Lista de columnas de clases (las etiquetas multilabel)
classes = [
    'papules', 'black_head', 'white_head', 'acne_scar', 'wrinkle', 'stains',
    'freckle', 'nodules', 'dark_circle', 'pustules', 'skinredness',
    'vascular', 'dark_spot', 'eye_bag'
]

# Dataset con augmentations aplicadas solo a clases minoritarias
train_dataset = CustomMultiLabelDataset(
    df=df_train,
    image_dir=f"{notebook_path}/train",
    classes=classes,
    transform=transform,
    aug_transform=aug_transform,
    minority_classes=minority_classes
)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nÉpoca {epoch+1}/{num_epochs}")
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    print(f"🔧 Train Loss: {train_loss:.4f}")


Época 1/10
🔧 Train Loss: 0.4134

Época 2/10
🔧 Train Loss: 0.1753

Época 3/10
🔧 Train Loss: 0.1249

Época 4/10
🔧 Train Loss: 0.1029

Época 5/10
🔧 Train Loss: 0.0779

Época 6/10
🔧 Train Loss: 0.0690

Época 7/10
🔧 Train Loss: 0.0602

Época 8/10
🔧 Train Loss: 0.0513

Época 9/10
🔧 Train Loss: 0.0439

Época 10/10
🔧 Train Loss: 0.0395


In [ ]:
evaluate_with_metrics(model, test_loader, device, class_names=classes)

Classification Report (per class):
              precision    recall  f1-score   support

     papules       0.00      0.00      0.00         2
  black_head       0.00      0.00      0.00         1
  white_head       0.00      0.00      0.00        15
   acne_scar       0.12      0.17      0.14         6
     wrinkle       0.00      0.00      0.00        21
      stains       0.06      0.14      0.09         7
     freckle       0.11      0.29      0.16         7
     nodules       0.00      0.00      0.00         3
 dark_circle       0.10      0.06      0.07        18
    pustules       0.00      0.00      0.00        16
 skinredness       0.00      0.00      0.00        19
    vascular       0.00      0.00      0.00        24
   dark_spot       0.00      0.00      0.00         0
     eye_bag       0.00      0.00      0.00        11

   micro avg       0.07      0.03      0.05       150
   macro avg       0.03      0.05      0.03       150
weighted avg       0.03      0.03      0.03  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Logistic Multilabel

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [ ]:
classes = [
    'papules', 'black_head', 'white_head', 'acne_scar', 'wrinkle', 'stains',
    'freckle', 'nodules', 'dark_circle', 'pustules', 'skinredness',
    'vascular', 'dark_spot', 'eye_bag'
]

In [ ]:
feature_extractor = models.efficientnet_b0(pretrained=True)
feature_extractor.classifier = torch.nn.Identity()
feature_extractor.eval().to(device)


simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

feature_dataset = CustomMultiLabelDataset(
    df=df_train,
    image_dir=f"{notebook_path}/train",
    classes=classes,
    transform=simple_transform,
    aug_transform=None,
    minority_classes=[]
)

feature_loader = DataLoader(feature_dataset, batch_size=32, shuffle=False)


features = []
targets = []

with torch.no_grad():
    for imgs, labels in feature_loader:
        imgs = imgs.to(device)
        feats = feature_extractor(imgs)
        features.append(feats.cpu().numpy())
        targets.append(labels.numpy())

X = np.vstack(features)
y = np.vstack(targets)


from sklearn.multioutput import MultiOutputClassifier

logistic = MultiOutputClassifier(LogisticRegression(max_iter=1000, class_weight='balanced'))
logistic.fit(X, y)


MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   max_iter=1000))

In [ ]:
test_dataset = CustomMultiLabelDataset(
    df=df_test,
    image_dir=f"{notebook_path}/valid",
    classes=classes,
    transform=simple_transform,
    aug_transform=None,
    minority_classes=[]
)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


X_test = []
y_test = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        feats = feature_extractor(imgs)
        X_test.append(feats.cpu().numpy())
        y_test.append(labels.numpy())

X_test = np.vstack(X_test)
y_test = np.vstack(y_test)



In [124]:
y_pred = logistic.predict(X_test)

print(classification_report(y_test, y_pred, target_names=classes))

              precision    recall  f1-score   support

     papules       0.58      0.67      0.62        21
  black_head       0.71      0.79      0.75        19
  white_head       0.62      0.54      0.58        24
   acne_scar       0.06      1.00      0.12         1
     wrinkle       0.86      1.00      0.92        18
      stains       0.77      0.91      0.83        11
     freckle       0.88      1.00      0.94        15
     nodules       0.20      0.33      0.25         6
 dark_circle       0.87      0.81      0.84        16
    pustules       0.12      0.14      0.13         7
 skinredness       0.60      0.86      0.71         7
    vascular       0.40      0.67      0.50         3
   dark_spot       0.00      0.00      0.00         0
     eye_bag       0.00      0.00      0.00         2

   micro avg       0.60      0.73      0.66       150
   macro avg       0.48      0.62      0.51       150
weighted avg       0.66      0.73      0.69       150
 samples avg       0.70   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [125]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score

subset_acc = accuracy_score(y_test, y_pred)
print(f"Subset Accuracy: {subset_acc:.4f}")

hamming = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {hamming:.4f}")

map = average_precision_score(y_test, y_pred)
print(f"MAP: {map:.4f}")

Subset Accuracy: 0.4950
Hamming Loss: 0.0806
MAP: 0.4111


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = MultiOutputClassifier(RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
))

rf.fit(X, y)

MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       n_jobs=-1,
                                                       random_state=42))

In [126]:
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf, target_names=classes))

              precision    recall  f1-score   support

     papules       0.60      0.29      0.39        21
  black_head       0.83      0.53      0.65        19
  white_head       0.60      0.12      0.21        24
   acne_scar       0.25      1.00      0.40         1
     wrinkle       0.94      0.89      0.91        18
      stains       0.60      0.27      0.38        11
     freckle       1.00      0.33      0.50        15
     nodules       0.00      0.00      0.00         6
 dark_circle       0.00      0.00      0.00        16
    pustules       0.00      0.00      0.00         7
 skinredness       0.00      0.00      0.00         7
    vascular       0.00      0.00      0.00         3
   dark_spot       0.00      0.00      0.00         0
     eye_bag       0.00      0.00      0.00         2

   micro avg       0.76      0.29      0.42       150
   macro avg       0.34      0.25      0.24       150
weighted avg       0.54      0.29      0.36       150
 samples avg       0.35   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [127]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score

subset_acc = accuracy_score(y_test, y_pred)
print(f"Subset Accuracy: {subset_acc:.4f}")

hamming = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {hamming:.4f}")

map = average_precision_score(y_test, y_pred)
print(f"MAP: {map:.4f}")

Subset Accuracy: 0.4950
Hamming Loss: 0.0806
MAP: 0.4111


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
knn = KNeighborsClassifier()
multi_knn = MultiOutputClassifier(knn)

param_grid = {
    'estimator__n_neighbors': [3, 5, 7, 10]
}

grid_search = GridSearchCV(multi_knn, param_grid, cv=3, scoring='f1_micro', verbose=1)
grid_search.fit(X, y)

# Mejor modelo
best_knn = grid_search.best_estimator_
print("Mejor valor de n_neighbors:", grid_search.best_params_)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Mejor valor de n_neighbors: {'estimator__n_neighbors': 3}


In [128]:
y_pred = best_knn.predict(X_test)

print("Reporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=classes))

Reporte de clasificación:
              precision    recall  f1-score   support

     papules       0.56      0.71      0.62        21
  black_head       0.57      0.63      0.60        19
  white_head       0.48      0.42      0.44        24
   acne_scar       0.06      1.00      0.12         1
     wrinkle       0.88      0.83      0.86        18
      stains       0.80      0.73      0.76        11
     freckle       0.83      0.67      0.74        15
     nodules       0.00      0.00      0.00         6
 dark_circle       1.00      0.38      0.55        16
    pustules       0.14      0.14      0.14         7
 skinredness       0.67      0.29      0.40         7
    vascular       0.00      0.00      0.00         3
   dark_spot       0.00      0.00      0.00         0
     eye_bag       0.00      0.00      0.00         2

   micro avg       0.56      0.53      0.54       150
   macro avg       0.43      0.41      0.37       150
weighted avg       0.62      0.53      0.55       150


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [129]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score

subset_acc = accuracy_score(y_test, y_pred)
print(f"Subset Accuracy: {subset_acc:.4f}")

hamming = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {hamming:.4f}")

map = average_precision_score(y_test, y_pred)
print(f"MAP: {map:.4f}")

Subset Accuracy: 0.4257
Hamming Loss: 0.0948
MAP: 0.2979


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


#XGBOOST

In [130]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_jobs=-1,
    verbosity=0
)


multi_xgb = MultiOutputClassifier(xgb_clf)
multi_xgb.fit(X, y)
y_pred = multi_xgb.predict(X_test)

print("Reporte de clasificación con XGBoost:")
print(classification_report(y_test, y_pred, target_names=classes))


Reporte de clasificación con XGBoost:
              precision    recall  f1-score   support

     papules       0.53      0.38      0.44        21
  black_head       0.79      0.58      0.67        19
  white_head       0.71      0.42      0.53        24
   acne_scar       0.12      1.00      0.22         1
     wrinkle       0.80      0.89      0.84        18
      stains       1.00      0.45      0.62        11
     freckle       0.83      0.67      0.74        15
     nodules       0.50      0.17      0.25         6
 dark_circle       1.00      0.31      0.48        16
    pustules       0.00      0.00      0.00         7
 skinredness       1.00      0.14      0.25         7
    vascular       0.00      0.00      0.00         3
   dark_spot       0.00      0.00      0.00         0
     eye_bag       0.00      0.00      0.00         2

   micro avg       0.70      0.45      0.55       150
   macro avg       0.52      0.36      0.36       150
weighted avg       0.72      0.45      0.5

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [131]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score

subset_acc = accuracy_score(y_test, y_pred)
print(f"Subset Accuracy: {subset_acc:.4f}")

hamming = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {hamming:.4f}")

map = average_precision_score(y_test, y_pred)
print(f"MAP: {map:.4f}")

Subset Accuracy: 0.3564
Hamming Loss: 0.0785
MAP: 0.2966


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [ ]:
import joblib

model_dir = '/content/drive/Shared drives/Desarrollo/models'

In [ ]:
joblib.dump(logistic, f"{model_dir}/logistic_regression_v1.pkl")
joblib.dump(rf, f"{model_dir}/random_forest_v1.pkl")
joblib.dump(best_knn, f"{model_dir}/knn_v1.pkl")
joblib.dump(multi_xgb, f"{model_dir}/xgboost_v1.pkl")

['/content/drive/Shared drives/Desarrollo/models/xgboost_v1.pkl']

In [ ]:
import os
os.makedirs(model_dir, exist_ok=True)

# MLP

In [115]:
from sklearn.neural_network import MLPClassifier

mlp_base = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    max_iter=300,
    random_state=42,
    verbose=True
)

mlp_clf = MultiOutputClassifier(mlp_base)


mlp_clf.fit(X, y)
y_pred = mlp_clf.predict(X_test)


print(classification_report(y_test, y_pred, target_names=classes, zero_division=0))

f1_macro = f1_score(y_test, y_pred, average='macro')

Iteration 1, loss = 0.67667764
Iteration 2, loss = 0.44497227
Iteration 3, loss = 0.37584669
Iteration 4, loss = 0.33425130
Iteration 5, loss = 0.30920942
Iteration 6, loss = 0.28979642
Iteration 7, loss = 0.27258615
Iteration 8, loss = 0.25205914
Iteration 9, loss = 0.24598701
Iteration 10, loss = 0.23138318
Iteration 11, loss = 0.21390178
Iteration 12, loss = 0.20408984
Iteration 13, loss = 0.18953629
Iteration 14, loss = 0.17880464
Iteration 15, loss = 0.16805769
Iteration 16, loss = 0.15810435
Iteration 17, loss = 0.14604168
Iteration 18, loss = 0.14655601
Iteration 19, loss = 0.13015259
Iteration 20, loss = 0.11972759
Iteration 21, loss = 0.11278869
Iteration 22, loss = 0.10217561
Iteration 23, loss = 0.09405451
Iteration 24, loss = 0.08819566
Iteration 25, loss = 0.08895190
Iteration 26, loss = 0.07249098
Iteration 27, loss = 0.06671567
Iteration 28, loss = 0.06542207
Iteration 29, loss = 0.06076472
Iteration 30, loss = 0.05217943
Iteration 31, loss = 0.04824385
Iteration 32, los

In [116]:
print(f"F1-macro: {f1_macro:.4f}")

F1-macro: 0.5330


In [117]:
print(classification_report(y_test, y_pred, target_names=classes, zero_division=0))

              precision    recall  f1-score   support

     papules       0.61      0.52      0.56        21
  black_head       0.74      0.74      0.74        19
  white_head       0.57      0.33      0.42        24
   acne_scar       0.08      1.00      0.15         1
     wrinkle       0.90      1.00      0.95        18
      stains       1.00      0.73      0.84        11
     freckle       0.87      0.87      0.87        15
     nodules       0.40      0.33      0.36         6
 dark_circle       0.92      0.69      0.79        16
    pustules       0.00      0.00      0.00         7
 skinredness       0.71      0.71      0.71         7
    vascular       0.50      0.33      0.40         3
   dark_spot       0.00      0.00      0.00         0
     eye_bag       1.00      0.50      0.67         2

   micro avg       0.68      0.62      0.65       150
   macro avg       0.59      0.55      0.53       150
weighted avg       0.71      0.62      0.65       150
 samples avg       0.67   

In [123]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score

subset_acc = accuracy_score(y_test, y_pred)
print(f"Subset Accuracy: {subset_acc:.4f}")

hamming = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {hamming:.4f}")

map = average_precision_score(y_test, y_pred)
print(f"MAP: {map:.4f}")

Subset Accuracy: 0.5347
Hamming Loss: 0.0714
MAP: 0.4300


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [119]:
joblib.dump(mlp_clf, f"{model_dir}/mlp_v1.pkl")

['/content/drive/Shared drives/Desarrollo/models/mlp_v1.pkl']

# Pruebas modelos clasicos

In [34]:
from PIL import Image

simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def preprocess_image_for_classical_model(image_path, scaler=None):
    image = Image.open(image_path).convert("RGB")
    image_tensor = simple_transform(image)
    image_array = image_tensor.numpy().reshape(-1)

    if scaler is not None:
        image_array = scaler.transform([image_array])[0]

    return image_array

In [36]:
def preprocess_image_for_classical_model(image_path):
    image = Image.open(image_path).convert('RGB')
    img_tensor = simple_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feats = feature_extractor(img_tensor)
    return feats.cpu().numpy().squeeze(0)

In [111]:
image_path = '/content/drive/Shared drives/Desarrollo/fotos de prueba/whac.jpg'

In [112]:
modelos=['mlp_v1.pkl', 'logistic_regression_v1.pkl', 'random_forest_v1.pkl', 'knn_v1.pkl', 'xgboost_v1.pkl']
for model_name in modelos:
  model_path = f"{model_dir}/{model_name}"
  model_loaded = joblib.load(model_path)
  x_new = preprocess_image_for_classical_model(image_path)
  y_pred = model_loaded.predict([x_new])[0]
  labels_predichas = [classes[i] for i, val in enumerate(y_pred) if val ==1]
  print("modelo", model_name)
  print("Predicciones:", labels_predichas)
  print("\n")

modelo mlp_v1.pkl
Predicciones: ['papules', 'black_head', 'freckle']


modelo logistic_regression_v1.pkl
Predicciones: ['papules', 'freckle']


modelo random_forest_v1.pkl
Predicciones: []


modelo knn_v1.pkl
Predicciones: []


modelo xgboost_v1.pkl
Predicciones: ['freckle']


